# Datos centrales de carbón arrancadas

Los datos de programación diaria de generación a nivel de central (UPROG) no se publican en la web pública de esios a diario, si no en la web de agentes, en el fichero P48_cierre. El dato se hace público al 90 día transcurrido.

Mi empresa, como agentes de mercado, dispone de esta información a diario y la almacenamos en sql. A través del siguiente código MySql, he descargado los datos calculando cuántas centrales había arrancadas cada día en cada hora y posteriormente lo he guardado en el .csv adjunto en el repositorio en el apartado datos.

~~~~sql
SELECT
      COUNT(DISTINCT(id_uprog)) AS NUM_PLANTAS
      ,format([FECHASESION],'yyyy-MM-dd') + ' ' + format([HORA]-1,'00') + ':00:00' AS datetime
      FROM [BBDD]  -- representa la bbdd de nuestro servidor
      where id_uprog in ('','',...)  -- lista de unidades. Existen 26 centrales de carbón activas en España
and fechasesion between '20150930' and '20180930'

GROUP BY datetime
 
~~~~

En este notebook leo los datos del .csv y los cargo en un nuevo dataframe.

## 1. Importamos librerías necesarias

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

## 2. Lectura y testeo de datos

In [2]:
carbon=pd.read_csv("../data/P48.csv",sep=";") #no pongo parse_dates=['datetime'] porque no sé el formato guardado
carbon.head() 

datetime  NUM_PLANTAS
0  30/9/15 0:00           22
1  30/9/15 1:00           22
2  30/9/15 2:00           22
3  30/9/15 3:00           23
4  30/9/15 4:00           23

Como feature no quiero las centrales de carbón arrancadas en el día, ya que no es un dato que tenga y pueda utilizar como entrada para predecir la subida/bajada del precio del intradiario (cuando debo apostar todavía no sé la casación ni qué centrales estarán).

Lo que utilizaré como input será qué centrales había arrancadas el día anterior. El hecho de que una central ya esté arrancada puede condicionar el comportamiento del intradiario al día siguiente en función del precio del MD resultante.

Por tanto, sumaré un día a datetime, de tal manera que para el día 01/10/2015 el dato que me muestre sea el nº de centrales de carbón arrancadas el 30/09/2015.

In [3]:
carbon.dtypes

datetime       object
NUM_PLANTAS     int64
dtype: object

In [4]:
#Convertimos a fecha
carbon['datetime']=pd.to_datetime(carbon['datetime'],dayfirst=True)

#Unificamos formato
carbon['datetime']=pd.to_datetime([x.strftime('%Y-%m-%d %H:%M:%S') for x in carbon['datetime']],yearfirst=True)
carbon.dtypes

datetime       datetime64[ns]
NUM_PLANTAS             int64
dtype: object

In [5]:
pd.to_datetime?

Signature: pd.to_datetime(arg, errors='raise', dayfirst=False, yearfirst=False, utc=None, box=True, format=None, exact=True, unit=None, infer_datetime_format=False, origin='unix', cache=False)
Docstring:
Convert argument to datetime.

Parameters
----------
arg : integer, float, string, datetime, list, tuple, 1-d array, Series

    .. versionadded:: 0.18.1

       or DataFrame/dict-like

errors : {'ignore', 'raise', 'coerce'}, default 'raise'

    - If 'raise', then invalid parsing will raise an exception
    - If 'coerce', then invalid parsing will be set as NaT
    - If 'ignore', then invalid parsing will return the input
dayfirst : boolean, default False
    Specify a date parse order if `arg` is str or its list-likes.
    If True, parses dates with the day first, eg 10/11/12 is parsed as
    2012-11-10.
    with day first (this is a known bug, based on dateutil behavior).
yearfirst : boolean, default False
    Specify a date parse order if `arg` is str or its list-likes.

    - If T

In [6]:
dia=timedelta(days=1)  #nº días que vamos a sumar para crear nuestra columna adicional

carbon['aux']=[x+dia for x in carbon['datetime']]
carbon.head()

datetime  NUM_PLANTAS                 aux
0 2015-09-30 00:00:00           22 2015-10-01 00:00:00
1 2015-09-30 01:00:00           22 2015-10-01 01:00:00
2 2015-09-30 02:00:00           22 2015-10-01 02:00:00
3 2015-09-30 03:00:00           23 2015-10-01 03:00:00
4 2015-09-30 04:00:00           23 2015-10-01 04:00:00

In [7]:
carbon.tail()

datetime  NUM_PLANTAS                 aux
26323 2018-09-30 19:00:00           13 2018-10-01 19:00:00
26324 2018-09-30 20:00:00           13 2018-10-01 20:00:00
26325 2018-09-30 21:00:00           13 2018-10-01 21:00:00
26326 2018-09-30 22:00:00           13 2018-10-01 22:00:00
26327 2018-09-30 23:00:00           13 2018-10-01 23:00:00

Las últimas 24 horas me sobran, mi histórico de datos (columna aux) lo he parado en 30/09/2018. 

Los últimos 24 datos serían los que ayudarían a predecir el mercado del día 01/10/2018.

In [8]:
#Seleccionamos las columnas que nos interesan
carbon_data=carbon[['aux','NUM_PLANTAS']]

In [9]:
#Eliminamos los datos del ultimo dia
ult_dia=pd.to_datetime('2018-10-01 00:00')

carbon_data2 = carbon_data[(carbon_data['aux']<ult_dia)]
carbon_data2.shape

(26304, 2)

In [10]:
#Revisamos duplicados. Al sumar 1 día a capón las horas duplicadas del día de 25 horas se traspasan al siguiente tambien.
#El día de 23h se habrá movido un dia

carbon_data2[carbon_data2['aux'].duplicated()==True]

aux  NUM_PLANTAS
603   2015-10-26 02:00:00           18
9507  2016-10-31 02:00:00           19
18243 2017-10-30 02:00:00           10

Al haber añadido 1 día, los días de 25 horas no me van a coincidir con los reales. Tendré que tenerlo en cuenta a la hora de unificar datos.

Lo mismo pasará con los de 23 h. Miro como ejemplo el día de 23h del 2018: '2018-03-25' y el siguiente:

In [11]:
carbon_data2[(carbon_data2['aux']>='2018-03-25') & (carbon_data2['aux']<'2018-03-27')]

aux  NUM_PLANTAS
21745 2018-03-25 00:00:00            4
21746 2018-03-25 01:00:00            4
21747 2018-03-25 02:00:00            4
21748 2018-03-25 03:00:00            4
21749 2018-03-25 04:00:00            4
21750 2018-03-25 05:00:00            4
21751 2018-03-25 06:00:00            4
21752 2018-03-25 07:00:00            4
21753 2018-03-25 08:00:00            4
21754 2018-03-25 09:00:00            4
21755 2018-03-25 10:00:00            4
21756 2018-03-25 11:00:00            4
21757 2018-03-25 12:00:00            4
21758 2018-03-25 13:00:00            4
21759 2018-03-25 14:00:00            4
21760 2018-03-25 15:00:00            4
21761 2018-03-25 16:00:00            4
21762 2018-03-25 17:00:00            4
21763 2018-03-25 18:00:00            4
21764 2018-03-25 19:00:00            4
21765 2018-03-25 20:00:00            4
21766 2018-03-25 21:00:00            4
21767 2018-03-25 22:00:00            4
21768 2018-03-25 23:00:00            4
21769 2018-03-26 00:00:00            4
21770 2018-03-26 01:00:00            4
21771 2018-03-26 02:00:00            4
21772 2018-03-26 03:00:00            4
21773 2018-03-26 04:00:00            4
21774 2018-03-26 05:00:00            4
21775 2018-03-26 06:00:00            4
21776 2018-03-26 07:00:00            4
21777 2018-03-26 08:00:00            4
21778 2018-03-26 09:00:00            4
21779 2018-03-26 10:00:00            4
21780 2018-03-26 11:00:00            4
21781 2018-03-26 12:00:00            4
21782 2018-03-26 13:00:00            4
21783 2018-03-26 14:00:00            4
21784 2018-03-26 15:00:00            4
21785 2018-03-26 16:00:00            4
21786 2018-03-26 17:00:00            4
21787 2018-03-26 18:00:00            4
21788 2018-03-26 19:00:00            4
21789 2018-03-26 20:00:00            4
21790 2018-03-26 21:00:00            4
21791 2018-03-26 22:00:00            4

Preparo .csv renombrando las columnas y asignando la fecha como índice.

In [12]:
carbon_dia_ant=carbon_data2.rename(columns={'aux':'datetime','NUM_PLANTAS':'NUM_PLANTAS_D-1'})
carbon_dia_ant.set_index('datetime',inplace=True)
carbon_dia_ant.head()

NUM_PLANTAS_D-1
datetime                            
2015-10-01 00:00:00               22
2015-10-01 01:00:00               22
2015-10-01 02:00:00               22
2015-10-01 03:00:00               23
2015-10-01 04:00:00               23

In [13]:
carbon_dia_ant.shape  #sin NaN

(26304, 1)

In [14]:
carbon_dia_ant['NUM_PLANTAS_D-1'].count()

26304

## 3. Guardamos en csv

In [15]:
carbon_dia_ant.to_csv('../data/carbon_dia_ant.csv',sep=';')